<a target="_blank" href="https://colab.research.google.com/github/okareo-ai/okareo-python-sdk/blob/main/examples/summarization_eval.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Welcome to Okareo!

Get your API token from [https://app.okareo.com/](https://app.okareo.com/) and set it in the cell below.
   (Note: You will also need an OpenAI key.)



In [ ]:
OKAREO_API_KEY = "<YOUR-OKAREO-API-KEY>"
OPENAI_API_KEY = "<YOUR-OPENAI-API-KEY>"

In [ ]:
%pip install okareo
%pip install openai

In [ ]:
# Simple summarization prompt using OpenAI'a GPT 3.5 Turbo model

from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

def get_turbo_summary(messages, model="gpt-3.5-turbo", 
  temperature=0, max_tokens=500):
  response = client.chat.completions.create(
    model=model,
    messages=messages,
    temperature=temperature, 
    max_tokens=max_tokens,
  )
  return response.choices[0].message.content

USER_PROMPT_TEMPLATE = "{input}"

SUMMARIZATION_CONTEXT_TEMPLATE = """
You will be provided with text.
Summarize the text in 1 or 2 simple sentences.
If the text's title and the author's name are available, refer to both in the summary.
Your goal is to make the summary clear to a middle school student.
"""

# Text for the summarization model to process
text_1 = """
I Wandered Lonely as a Cloud

I wandered lonely as a cloud
That floats on high o'er vales and hills,
When all at once I saw a crowd,
A host, of golden daffodils;
Beside the lake, beneath the trees,
Fluttering and dancing in the breeze.

Continuous as the stars that shine
And twinkle on the milky way,
They stretched in never-ending line
Along the margin of a bay:
Ten thousand saw I at a glance,
Tossing their heads in sprightly dance.

The waves beside them danced; but they
Out-did the sparkling waves in glee:
A poet could not but be gay,
In such a jocund company:
I gazed-and gazed-but little thought
What wealth the show to me had brought:

For oft, when on my couch I lie
In vacant or in pensive mood,
They flash upon that inward eye
Which is the bliss of solitude;
And then my heart with pleasure fills,
And dances with the daffodils.

-- by William Wordsworth
"""

text_2 = """
All Things Are Current Found

ALL things are current found
On earthly ground,
Spirits and elements
Have their descents.

Night and day, year on year,
High and low, far and near,
These are our own aspects,
These are our own regrets.

Ye gods of the shore,
Who abide evermore,
I see you far headland,
Stretching on either hand;

I hear the sweet evening sounds
From your undecaying grounds;
Cheat me no more with time,
Take me to your clime.

-- by Henry D. Thoreau
"""

text_3 = """
Hope Is the Thing with Feathers

"Hope" is the thing with feathers-
That perches in the soul-
And sings the tune without the words-
And never stops-at all-

And sweetest-in the Gale-is heard-
And sore must be the storm-
That could abash the little Bird
That kept so many warm-

I've heard it in the chillest land-
And on the strangest Sea-
Yet, never, in Extremity,
It asked a crumb-of Me.

-- by Emily Dickinson
"""

In [ ]:
# Create a scenario to evaluate the model with
import uuid
import random
import string
from okareo import Okareo
from okareo_api_client.models import ScenarioSetCreate, ScenarioSetResponse, SeedData, ScenarioType

random_string = ''.join(random.choices(string.ascii_letters, k=5))
okareo = Okareo(OKAREO_API_KEY)

scenario_set_create = ScenarioSetCreate(
    name=f"Summarization - {random_string}",
    number_examples=1,
    generation_type=ScenarioType.SEED,
    seed_data=[
        SeedData(
            input_="Paul Merson has restarted his row with Andros Townsend after the Tottenham midfielder was brought on with only seven minutes remaining in his team 's 0-0 draw with Burnley on Sunday . 'Just been watching the game , did you miss the coach ? # RubberDub # 7minutes , ' Merson put on Twitter . Merson initially angered Townsend for writing in his Sky Sports column that 'if Andros Townsend can get in ( the England team ) then it opens it up to anybody . ' Paul Merson had another dig at Andros Townsend after his appearance for Tottenham against Burnley Townsend was brought on in the 83rd minute for Tottenham as they drew 0-0 against Burnley Andros Townsend scores England 's equaliser in their 1-1 friendly draw with Italy in Turin on Tuesday night The former Arsenal man was proven wrong when Townsend hit a stunning equaliser for England against Italy and he duly admitted his mistake . 'It 's not as though I was watching hoping he would n't score for England , I 'm genuinely pleased for him and fair play to him – it was a great goal , ' Merson said . 'It 's just a matter of opinion , and my opinion was that he got pulled off after half an hour at Manchester United in front of Roy Hodgson , so he should n't have been in the squad . 'When I 'm wrong , I hold my hands up . I do n't have a problem with doing that - I 'll always be the first to admit when I 'm wrong . ' Townsend hit back at Merson on Twitter after scoring for England against Italy Sky Sports pundit Merson ( centre ) criticised Townsend 's call-up to the England squad last week Townsend hit back at Merson after netting for England in Turin on Wednesday , saying 'Not bad for a player that should be 'nowhere near the squad ' ay @ PaulMerse ? ' Any bad feeling between the pair seemed to have passed but Merson was unable to resist having another dig at Townsend after Tottenham drew at Turf Moor .",  
            result=str(uuid.uuid4())
        ),
        SeedData(
            input_=text_2,  
            result=str(uuid.uuid4())
        ),
        SeedData(
            input_=text_3, 
            result=str(uuid.uuid4())
        )
    ],
)
scenario = okareo.create_scenario_set(scenario_set_create)

In [ ]:
# Evaluate the scenario and model combination and then get a link to the results on Okareo

import random
import string
from okareo import Okareo
from okareo_api_client.models import ScenarioSetCreate, ScenarioSetResponse, SeedData
from okareo.model_under_test import OpenAIModel
from okareo_api_client.models.test_run_type import TestRunType

okareo = Okareo(OKAREO_API_KEY)
random_string = ''.join(random.choices(string.ascii_letters, k=5))
mut_name = f"OpenAI Summarization Model - {random_string}"
eval_name = f"Summarization Run - {random_string}"

model_under_test = okareo.register_model(
    name=mut_name,
    model=OpenAIModel(
        model_id="gpt-3.5-turbo",
        temperature=0,
        system_prompt_template=SUMMARIZATION_CONTEXT_TEMPLATE,
        user_prompt_template=USER_PROMPT_TEMPLATE,
    ),
)

evaluation = model_under_test.run_test(
    name=eval_name,
    scenario=scenario,
    api_key=OPENAI_API_KEY,
    test_run_type=TestRunType.NL_GENERATION,
    calculate_metrics=True,
)

print(f"See results in Okareo: {evaluation.app_link}")